In [4]:
import pandas as pd
from filter_utlis import filter_exercise
import matplotlib.pyplot as plt
import numpy as np

In [10]:
df = pd.read_csv("large_small_grasp.csv")

In [12]:
large_grasp_df = filter_exercise(df,[0,18])
small_grasp_df = filter_exercise(df,[0,19])

In [13]:
def segment_repetitions(df, grasp_label):
    """Segments a dataframe into a list of individual grasp repetitions."""
    repetitions = []
    # Find the indices where a grasp starts (stimulus changes from 0 to the label)
    start_indices = df[(df['stimulus'].shift(1) == 0) & (df['stimulus'] == grasp_label)].index

    for start in start_indices:
        # Find the end of this specific grasp (when it returns to 0)
        end_index_series = df.index[ (df.index > start) & (df['stimulus'] == 0) ]
        if not end_index_series.empty:
            end = end_index_series[0]
            # Extract the window for this single repetition
            repetition_df = df.loc[start:end]
            repetitions.append(repetition_df)
            
    return repetitions

In [14]:
def calculate_features(window):
    """Calculates a dictionary of features for a given signal window."""
    features = {
        'rms': np.sqrt(np.mean(window**2)),
        'mav': np.mean(np.abs(window)),
        'zc': ((window[:-1] * window[1:]) < 0).sum(), # Requires numpy array conversion
        'wl': np.sum(np.abs(np.diff(window)))
    }
    return features

In [15]:
# Segment both large and small grasps
large_grasp_reps = segment_repetitions(large_grasp_df, 18)
small_grasp_reps = segment_repetitions(small_grasp_df, 19)

print(f"Found {len(large_grasp_reps)} repetitions of large grasps.")
print(f"Found {len(small_grasp_reps)} repetitions of small grasps.")

Found 240 repetitions of large grasps.
Found 240 repetitions of small grasps.


In [16]:
feature_list = []
emg_channels = [str(i) for i in range(12)] # Columns '0' through '11'

# Process large grasps
for rep in large_grasp_reps:
    rep_features = {}
    for channel in emg_channels:
        # Ensure data is numpy array for calculations
        channel_data = rep[channel].to_numpy()
        calculated = calculate_features(channel_data)
        for feature_name, value in calculated.items():
            rep_features[f'ch{channel}_{feature_name}'] = value
    
    rep_features['label'] = 'large_grasp'
    feature_list.append(rep_features)

# Process small grasps
for rep in small_grasp_reps:
    rep_features = {}
    for channel in emg_channels:
        channel_data = rep[channel].to_numpy()
        calculated = calculate_features(channel_data)
        for feature_name, value in calculated.items():
            rep_features[f'ch{channel}_{feature_name}'] = value
            
    rep_features['label'] = 'small_grasp'
    feature_list.append(rep_features)

# Create the final feature dataframe
feature_df = pd.DataFrame(feature_list)

In [17]:
feature_df.to_csv('feature_df.csv',index=False)